In [1]:
import argparse
import logging

import yaml

parser = argparse.ArgumentParser(description="GAT")
parser.add_argument(
    "--task",
    type=str,
    default="classification",
    choices=["classification", "clustering"],
)
parser.add_argument("--dataset", type=str, default="acm")
args, _ = parser.parse_known_args()


def load_best_configs(args, path):
    with open(path, "r") as f:
        configs = yaml.load(f, yaml.FullLoader)

    if args.dataset not in configs:
        logging.info("Best args not found")
        return args

    configs = configs[args.dataset]

    for k, v in configs.items():
        if "lr" in k or "weight_decay" in k:
            v = float(v)
        setattr(args, k, v)
    print("------ Use best configs ------")
    return args

In [80]:
parser = argparse.ArgumentParser(description="GAT")
parser.add_argument(
    "--task",
    type=str,
    default="classification",
    choices=["classification", "clustering"],
)
parser.add_argument("--dataset", type=str, default="acm")
args, _ = parser.parse_known_args()

args = load_best_configs(args, "./configs.yml")
args.__dict__

------ Use best configs ------


{'task': 'classification',
 'dataset': 'acm',
 'category': 'paper',
 'activation': 'prelu',
 'alpha_l': 1,
 'attn_drop': 0.5,
 'decoder': 'han',
 'encoder': 'han',
 'eva_lr': 0.01,
 'eva_wd': 0.0005,
 'feat_drop': 0.2,
 'feat_mask_rate': '0.5,0.005,0.8',
 'gpu': 0,
 'in_dim': 1902,
 'hidden_dim': 256,
 'l2_coef': 0,
 'leave_unchanged': 0.3,
 'loss_func': 'sce',
 'lr': 0.0008,
 'mae_epochs': 3,
 'mask_rate': 0.4,
 'mp2vec_feat_alpha_l': 2,
 'mp2vec_feat_drop': 0.2,
 'mp2vec_feat_pred_loss_weight': 0.1,
 'mp_edge_alpha_l': 3,
 'mp_edge_mask_rate': 0.7,
 'mp_edge_recon_loss_weight': 1,
 'mps_batch_size': 256,
 'mps_context_size': 3,
 'mps_embedding_dim': 64,
 'mps_epoch': 1,
 'mps_lr': 0.001,
 'mps_num_negative_samples': 3,
 'mps_walk_length': 10,
 'mps_walks_per_node': 3,
 'n_labels': 3,
 'negative_slope': 0.2,
 'nei_num': 2,
 'norm': 'batchnorm',
 'num_heads': 4,
 'num_layers': 2,
 'num_out_heads': 1,
 'optimizer': 'adam',
 'patience': 10,
 'replace_rate': 0.2,
 'residual': False,
 'sch

In [3]:
from openhgnn.dataset.NodeClassificationDataset import OHGB_NodeClassification

E:\Anacodna3\envs\GML\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
acm = OHGB_NodeClassification("ohgbn-acm", raw_dir="./dataset", logger="")

Extracting file to ./openhgnn/dataset\ohgbn-acm
Done saving data into cached files.


In [5]:
hg = acm.g
metapaths_dict = acm.meta_paths_dict

In [183]:
import importlib

import dgl
import hgmae
import torch

importlib.reload(hgmae)
from hgmae import HGMAE
HGMAE?

Init signature:
HGMAE(
    metapaths_dict,
    category,
    in_dim,
    hidden_dim,
    num_layers,
    num_heads,
    num_out_heads,
    feat_drop,
    attn_drop,
    negative_slope=0.2,
    residual=False,
    activation='prelu',
    norm=None,
    concat_out=True,
    loss_func='sce',
    use_mp_edge_recon=True,
    mp_edge_recon_loss_weight=1,
    mp_edge_mask_rate=0.7,
    mp_edge_alpha_l=3,
)
Docstring:     
Parameter
----------
metapaths_dict: dict[str, list[etype]]
    Dict from meta path name to meta path
category : string
    The category of the nodes to be classificated
in_dim:
hidden_dim: dim of encoded embedding
num_layers: int
    Number of layers of HAN encoder and decoder
num_heads: int
    Number of attention heads of hidden layers in HAN encoder and decoder
num_out_heads: int
    Number of attention heads of output projection in HAN encoder and decoder
activation: str or None
    Activation function, str in {'relu','gelu','prelu','elu'} or None, default: ``'prelu'``


In [179]:
h_dict=hg.ndata['h']

In [180]:
model = HGMAE.build_model_from_args(args, metapaths_dict)



In [181]:
model.forward(hg,h_dict,3)

tensor(0.1739, grad_fn=<MulBackward0>)

In [58]:
mps=[dgl.metapath_reachable_graph(hg,mp).adjacency_matrix() for mp in metapaths_dict.values()]

In [59]:
gs = []
for mp in mps:
    indices = mp.indices()
    cur_graph = dgl.graph((indices[0], indices[1]))
    gs.append(cur_graph)


In [224]:
l=torch.arange(15)

a=4
b=5
c=6
l

tensor([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14])

In [229]:
l[:a]

tensor([0, 1, 2, 3])

In [234]:
l[a:a+b]

tensor([4, 5, 6, 7, 8])

In [235]:
l[a+b:]

tensor([ 9, 10, 11, 12, 13, 14])